In [1]:
import os
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import soundfile as sf  # Library to save .wav files
from scipy.signal import butter, filtfilt
import pandas as pd
import librosa
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import hilbert, find_peaks

In [ ]:

############# Filted using butterworth #############


import os
import librosa
import numpy as np
import matplotlib.pyplot as plt
import soundfile as sf  # Library to save .wav files
from scipy.signal import butter, filtfilt

# Butterworth high-pass filter function
def butter_highpass(cutoff, sample_rate, order=4):
    nyquist = 0.5 * sample_rate
    normal_cutoff = cutoff / nyquist
    b, a = butter(order, normal_cutoff, btype='high', analog=False)
    return b, a

def apply_butter_highpass(samples, sample_rate, cutoff=50, order=4):
    b, a = butter_highpass(cutoff, sample_rate, order=order)
    filtered_samples = filtfilt(b, a, samples)
    return filtered_samples

# Define the path to the folder containing .wav files
input_folder_path = "D:/Example"


# Iterate through all files in the folder
for file in os.listdir(input_folder_path):
    if file.endswith('.wav'):
        # Full file paths
        file_path = os.path.join(input_folder_path, file)
        

        # Load the audio file using librosa
        samples, sample_rate = librosa.load(file_path, sr=None)

        # Apply Butterworth high-pass filter
        filtered_samples = apply_butter_highpass(samples, sample_rate)

       

        # Plot the original and filtered signals
        plt.figure(figsize=(14, 6))

        # Plot the original waveform
        plt.subplot(2, 1, 1)
        plt.plot(np.arange(len(samples)) / sample_rate, samples, label='Original Signal', color='b')
        plt.title(f"Original Phonocardiogram - {file}")
        plt.xlabel('Time [seconds]')
        plt.ylabel('Amplitude')
        plt.grid()

        # Plot the filtered waveform
        plt.subplot(2, 1, 2)
        plt.plot(np.arange(len(filtered_samples)) / sample_rate, filtered_samples, label='Filtered Signal', color='r')
        plt.title(f"Filtered Phonocardiogram (Butterworth High-Pass) - {file}")
        plt.xlabel('Time [seconds]')
        plt.ylabel('Amplitude')
        plt.grid()

        plt.tight_layout()
        plt.show()

In [ ]:


############  finding S1,S2 peaks according to the annotations  #############


# Define paths
wav_folder_path = "D:\ISI internship\Saved_from_Filter\WAV_Files"
tsv_folder_path = "D:\ISI internship\Saved_from_Filter\TSV_Files"
hea_folder_path = "D:\ISI internship\Saved_from_Filter\HEA_Files"

# Function to read the .hea file and extract metadata
def read_hea_file(hea_file_path):
    metadata = {}
    with open(hea_file_path, 'r') as file:
        first_line = file.readline().strip().split()
        metadata['record_name'] = first_line[0]
        metadata['channels'] = int(first_line[1])
        metadata['sample_rate'] = int(first_line[2])
        metadata['samples'] = int(first_line[3])
    return metadata

# Function to extract the envelope using Hilbert Transform
def extract_envelope(signal):
    analytic_signal = hilbert(signal)
    envelope = np.abs(analytic_signal)
    return envelope

# Function to segment heart sounds
def segment_heart_sounds(signal, sample_rate):
    envelope = extract_envelope(signal)
    peaks, _ = find_peaks(envelope, height=np.max(envelope) * 0.3, distance=0.3 * sample_rate)

    systole = []
    diastole = []
    
    for i in range(1, len(peaks) - 1, 2):
        s1 = peaks[i - 1]
        s2 = peaks[i]
        systole.append(signal[s1:s2])
        if i + 1 < len(peaks):
            next_s1 = peaks[i + 1]
            diastole.append(signal[s2:next_s1])
    
    return peaks, systole, diastole

# Process each .wav file in the folder
for file in os.listdir(wav_folder_path):
    if file.endswith('.wav'):
        wav_file_path = os.path.join(wav_folder_path, file)
        hea_file_path = os.path.join(hea_folder_path, file.replace('.wav', '.hea'))

        # Read .hea file to get metadata
        hea_metadata = read_hea_file(hea_file_path)
        print(f"Loaded .hea file with metadata: {hea_metadata}")

        # Load the .wav file using the sample rate from the .hea file
        samples, sample_rate = librosa.load(wav_file_path, sr=hea_metadata['sample_rate'])

        # Segment the heart sounds
        peaks, systole, diastole = segment_heart_sounds(samples, sample_rate)

        # Plotting results
        plt.figure(figsize=(14, 6))
        time_axis = np.arange(len(samples)) / sample_rate
        plt.plot(time_axis, samples, label='Heart Sound Signal', color='b')
        plt.scatter(peaks / sample_rate, samples[peaks], color='r', label='Detected Peaks')
        
        plt.title(f"Heart Sound Segmentation - {file}")
        plt.xlabel("Time (seconds)")
        plt.ylabel("Amplitude")
        plt.legend()
        plt.grid()
        plt.show()


In [ ]:


############ Total Event #################


# Define paths
wav_folder_path = "D:\\ISI internship\\Saved_from_Filter\\WAV_Files"
tsv_folder_path = "D:\\ISI internship\\Saved_from_Filter\\TSV_Files"
hea_folder_path = "D:\\ISI internship\\Saved_from_Filter\\HEA_Files"

# Function to read the .hea file and extract metadata
def read_hea_file(hea_file_path):
    metadata = {}
    with open(hea_file_path, 'r') as file:
        first_line = file.readline().strip().split()
        metadata['record_name'] = first_line[0]
        metadata['channels'] = int(first_line[1])
        metadata['sample_rate'] = int(first_line[2])
        metadata['samples'] = int(first_line[3])
    return metadata

# Function to extract the envelope using Hilbert Transform
def extract_envelope(signal):
    analytic_signal = hilbert(signal)
    envelope = np.abs(analytic_signal)
    return envelope

# Function to segment heart sounds
def segment_heart_sounds(signal, sample_rate):
    envelope = extract_envelope(signal)
    peaks, _ = find_peaks(envelope, height=np.max(envelope) * 0.3, distance=0.3 * sample_rate)
    return peaks

# Process each .wav file in the folder
for file in os.listdir(wav_folder_path):
    if file.endswith('.wav'):
        wav_file_path = os.path.join(wav_folder_path, file)
        hea_file_path = os.path.join(hea_folder_path, file.replace('.wav', '.hea'))
        tsv_file_path = os.path.join(tsv_folder_path, file.replace('.wav', '.tsv'))

        # Read .hea file to get metadata
        hea_metadata = read_hea_file(hea_file_path)
        print(f"Loaded .hea file with metadata: {hea_metadata}")

        # Load the .wav file using the sample rate from the .hea file
        samples, sample_rate = librosa.load(wav_file_path, sr=hea_metadata['sample_rate'])

        # Segment the heart sounds
        peaks = segment_heart_sounds(samples, sample_rate)

        # Load the .tsv file for annotations
        tsv_data = pd.read_csv(tsv_file_path, sep='\t', header=None, names=['start', 'end', 'event'])

        # Plotting results
        plt.figure(figsize=(14, 6))
        time_axis = np.arange(len(samples)) / sample_rate
        plt.plot(time_axis, samples, label='Heart Sound Signal', color='b')
        plt.scatter(peaks / sample_rate, samples[peaks], color='r', label='Detected Peaks')

        # Add labels to the peaks
        for idx, peak in enumerate(peaks):
            plt.text(peak / sample_rate, samples[peak], f'Peak {idx + 1}', color='red', fontsize=10, ha='center', va='bottom')

        # Plot annotations from the TSV file
        for idx, row in tsv_data.iterrows():
            start_time = row['start']
            end_time = row['end']
            event_label = row['event']
            plt.axvspan(start_time, end_time, color='c', alpha=0.3, label=f'Event {event_label}' if idx == 0 else "")

        plt.title(f"Heart Sound Segmentation - {file}")
        plt.xlabel("Time (seconds)")
        plt.ylabel("Amplitude")
        plt.legend()
        plt.grid()
        plt.show()


In [ ]:

################ S1,S2, SYSTOLE, DYSTOLE visualization ###############


# Define paths
wav_folder_path = "D:\\ISI internship\\Saved_from_Filter\\WAV_Files"
tsv_folder_path = "D:\\ISI internship\\Saved_from_Filter\\TSV_Files"
hea_folder_path = "D:\\ISI internship\\Saved_from_Filter\\HEA_Files"
output_folder = "D:\ISI internship\Saved_from_Filter\Plots_Segmentation"

# Ensure output directory exists
os.makedirs(output_folder, exist_ok=True)

# Function to read the .hea file and extract metadata
def read_hea_file(hea_file_path):
    metadata = {}
    with open(hea_file_path, 'r') as file:
        first_line = file.readline().strip().split()
        metadata['record_name'] = first_line[0]
        metadata['channels'] = int(first_line[1])
        metadata['sample_rate'] = int(first_line[2])
        metadata['samples'] = int(first_line[3])
    return metadata

# Function to extract the envelope using Hilbert Transform
def extract_envelope(signal):
    analytic_signal = hilbert(signal)
    envelope = np.abs(analytic_signal)
    return envelope

# Function to segment heart sounds
def segment_heart_sounds(signal, sample_rate):
    envelope = extract_envelope(signal)
    peaks, _ = find_peaks(envelope, height=np.max(envelope) * 0.3, distance=0.3 * sample_rate)
    return peaks

# Color mapping for events
color_map = {
    1: 'red',     # S1
    2: 'green',   # S2
    3: 'yellow',  # Systole
    4: 'blue',    # Diastole
    0: 'black'    # Default event
}

# Process each .wav file in the folder
for file in os.listdir(wav_folder_path):
    if file.endswith('.wav'):
        wav_file_path = os.path.join(wav_folder_path, file)
        hea_file_path = os.path.join(hea_folder_path, file.replace('.wav', '.hea'))
        tsv_file_path = os.path.join(tsv_folder_path, file.replace('.wav', '.tsv'))

        # Read .hea file to get metadata
        hea_metadata = read_hea_file(hea_file_path)
        print(f"Loaded .hea file with metadata: {hea_metadata}")

        # Load the .wav file using the sample rate from the .hea file
        samples, sample_rate = librosa.load(wav_file_path, sr=hea_metadata['sample_rate'])

        # Segment the heart sounds
        peaks = segment_heart_sounds(samples, sample_rate)

        # Load the .tsv file for annotations
        tsv_data = pd.read_csv(tsv_file_path, sep='\t', header=None, names=['start', 'end', 'event'])

        # Plotting results
        plt.figure(figsize=(14, 6))
        time_axis = np.arange(len(samples)) / sample_rate
        plt.plot(time_axis, samples, label='Heart Sound Signal', color='b')
        plt.scatter(peaks / sample_rate, samples[peaks], color='r', label='Detected Peaks')

        # Add labels to the peaks
        for idx, peak in enumerate(peaks):
            plt.text(peak / sample_rate, samples[peak], f'Peak {idx + 1}', color='red', fontsize=10, ha='center', va='bottom')

        # Plot annotations from the TSV file with color coding
        for idx, row in tsv_data.iterrows():
            start_time = row['start']
            end_time = row['end']
            event_label = row['event']
            color = color_map.get(int(event_label), 'black')  # Default to black if not found
            plt.axvspan(start_time, end_time, color=color, alpha=0.3, label=f'Event {event_label}' if idx == 0 else "")

        # Adding legend for events
        handles = [
            plt.Line2D([0], [0], color='red', lw=4, label='S1'),
            plt.Line2D([0], [0], color='green', lw=4, label='S2'),
            plt.Line2D([0], [0], color='yellow', lw=4, label='Systole'),
            plt.Line2D([0], [0], color='blue', lw=4, label='Diastole'),
            plt.Line2D([0], [0], color='black', lw=4, label='Default Event (0.0)')
        ]
        plt.legend(handles=handles)

        plt.title(f"Heart Sound Segmentation - {file}")
        plt.xlabel("Time (seconds)")
        plt.ylabel("Amplitude")
        plt.grid()
        
        # Save the figure
        output_file_path = os.path.join(output_folder, f"{file.replace('.wav', '.png')}")
        plt.savefig(output_file_path)
        plt.close()  # Close the figure to free up memory


In [8]:
pip install pandas librosa openpyxl

Note: you may need to restart the kernel to use updated packages.


In [6]:
%pip install pyfftw

Note: you may need to restart the kernel to use updated packages.


In [ ]:

############# Feature Extraction ##################


# Define paths
wav_folder_path = "D:\\ISI internship\\Saved_from_Filter\\WAV_Files"
hea_folder_path = "D:\\ISI internship\\Saved_from_Filter\\HEA_Files"
output_excel_path = "D:\\ISI internship\\Saved_from_Filter\\Extracted_Features.xlsx"  # Path to save the Excel file

# Function to read the .hea file and extract metadata
def read_hea_file(hea_file_path):
    metadata = {}
    with open(hea_file_path, 'r') as file:
        first_line = file.readline().strip().split()
        metadata['record_name'] = first_line[0]
        metadata['channels'] = int(first_line[1])
        metadata['sample_rate'] = int(first_line[2])
        metadata['samples'] = int(first_line[3])
    return metadata

# Function to extract the envelope using Hilbert Transform
def extract_envelope(signal):
    analytic_signal = hilbert(signal)
    envelope = np.abs(analytic_signal)
    return envelope

# Function to segment heart sounds
def segment_heart_sounds(signal, sample_rate):
    envelope = extract_envelope(signal)
    peaks, _ = find_peaks(envelope, height=np.max(envelope) * 0.3, distance=0.3 * sample_rate)
    return peaks

# Function to extract features from the audio signal
def extract_features(signal, sample_rate):
    features = {}

    # Dynamically set n_fft based on the length of the signal

    n_fft = min(2048, len(signal))  # Ensure n_fft is not larger than the signal length

    # Time-domain features


    features['envelope'] = np.mean(extract_envelope(signal))  # Envelope
    features['amplitude'] = np.mean(signal)  # Amplitude
    features['energy'] = np.sum(signal ** 2)  # Energy
    features['entropy'] = -np.sum(np.square(signal) * np.log(np.square(signal) + 1e-10))  # Entropy
    features['loudness'] = np.mean(np.abs(signal))  # Loudness
    features['peak_amplitude'] = np.max(np.abs(signal))  # Peak Amplitude

    # Spectral features


    spectral_centroid = librosa.feature.spectral_centroid(y=signal, sr=sample_rate, n_fft=n_fft)
    features['spectral_centroid'] = np.mean(spectral_centroid)  # Spectral Centroid

    return features
    
# List to hold all extracted features
all_features = []

# Process each .wav file in the folder
for file in os.listdir(wav_folder_path):
    if file.endswith('.wav'):
        wav_file_path = os.path.join(wav_folder_path, file)
        hea_file_path = os.path.join(hea_folder_path, file.replace('.wav', '.hea'))

        # Check if the .hea file exists
        if not os.path.exists(hea_file_path):
            print(f"Warning: HEA file not found for {file}. Skipping this file.")
            continue

        # Read .hea file to get metadata
        hea_metadata = read_hea_file(hea_file_path)
        print(f"Loaded .hea file with metadata: {hea_metadata}")

        # Load the .wav file using the sample rate from the .hea file
        try:
            samples, sample_rate = librosa.load(wav_file_path, sr=hea_metadata['sample_rate'])
        except Exception as e:
            print(f"Error loading {file}: {e}. Skipping this file.")
            continue

        # Segment the heart sounds
        peaks = segment_heart_sounds(samples, sample_rate)

        # Extract features for each segment
        for peak in peaks:
            start_idx = max(0, peak - int(0.1 * sample_rate))  # 100 ms before peak
            end_idx = min(len(samples), peak + int(0.1 * sample_rate))  # 100 ms after peak
            segment = samples[start_idx:end_idx]

            features = extract_features(segment, sample_rate)
            features['filename'] = file  # Add filename for reference
            all_features.append(features)  # Append to the list

# Create a DataFrame from the collected features
features_df = pd.DataFrame(all_features)

# Save to Excel
features_df.to_excel(output_excel_path, index=False)
print(f"Features saved to {output_excel_path}")


In [ ]:


################# Denoisation of soundwaves ##################


import os
import numpy as np
import pandas as pd
import librosa
import pywt
import soundfile as sf  # Import soundfile for saving WAV files

# Define paths
wav_folder_path = "D:\\ISI internship\\Saved_from_Filter\\WAV_Files"
hea_folder_path = "D:\\ISI internship\\Saved_from_Filter\\HEA_Files"
tsv_folder_path = "D:\\ISI internship\\Saved_from_Filter\\TSV_Files"
output_folder_path = "D:\\ISI internship\\Saved_from_Filter\\Denoised_WAV_Files"  # Path to save denoised files

# Create output folder if it doesn't exist
os.makedirs(output_folder_path, exist_ok=True)

# Function to read the .hea file and extract metadata
def read_hea_file(hea_file_path):
    metadata = {}
    with open(hea_file_path, 'r') as file:
        first_line = file.readline().strip().split()
        metadata['record_name'] = first_line[0]
        metadata['channels'] = int(first_line[1])
        metadata['sample_rate'] = int(first_line[2])
        metadata['samples'] = int(first_line[3])
    return metadata

# Function to read the .tsv file and extract segments (S1, S2, systole, diastole)
def read_tsv_file(tsv_file_path):
    tsv_data = pd.read_csv(tsv_file_path, sep='\t', header=None)
    events = {
        'start_time': tsv_data[0].values,
        'end_time': tsv_data[1].values,
        'label': tsv_data[2].values  # Assuming the 3rd column contains labels like S1, S2, systole, diastole
    }
    return events

# Function for wavelet denoising
def wavelet_denoise(signal, wavelet='db4', level=4):
    coeffs = pywt.wavedec(signal, wavelet, level=level)
    # Thresholding - can be adjusted
    threshold = np.sqrt(2 * np.log(len(signal))) * np.median(np.abs(coeffs[-1]))  # Universal threshold
    coeffs_denoised = [pywt.threshold(c, threshold, mode='soft') for c in coeffs]
    denoised_signal = pywt.waverec(coeffs_denoised, wavelet)
    return denoised_signal[:len(signal)]  # Match the original length

# Process each .wav file in the folder
for file in os.listdir(wav_folder_path):
    if file.endswith('.wav'):
        wav_file_path = os.path.join(wav_folder_path, file)
        hea_file_path = os.path.join(hea_folder_path, file.replace('.wav', '.hea'))
        tsv_file_path = os.path.join(tsv_folder_path, file.replace('.wav', '.tsv'))

        # Check if the .hea and .tsv files exist
        if not os.path.exists(hea_file_path):
            print(f"Warning: HEA file not found for {file}. Skipping this file.")
            continue
        if not os.path.exists(tsv_file_path):
            print(f"Warning: TSV file not found for {file}. Skipping this file.")
            continue

        # Read .hea file to get metadata
        hea_metadata = read_hea_file(hea_file_path)
        print(f"Loaded .hea file with metadata: {hea_metadata}")

        # Load the .wav file using the sample rate from the .hea file
        try:
            samples, sample_rate = librosa.load(wav_file_path, sr=hea_metadata['sample_rate'])
        except Exception as e:
            print(f"Error loading {file}: {e}. Skipping this file.")
            continue

        # Read the .tsv file for event timings (e.g., S1, S2)
        events = read_tsv_file(tsv_file_path)

        # Denoise each segment based on .tsv timings and save the denoised audio
        for start_time, end_time, label in zip(events['start_time'], events['end_time'], events['label']):
            start_sample = int(start_time * sample_rate)
            end_sample = int(end_time * sample_rate)
            segment = samples[start_sample:end_sample]

            # Check if the segment is valid
            if len(segment) > 0:
                # Denoise the segment
                denoised_segment = wavelet_denoise(segment)

                # Save the denoised segment as a new .wav file
                denoised_file_name = f"{os.path.splitext(file)[0]}_{label}.wav"
                denoised_file_path = os.path.join(output_folder_path, denoised_file_name)
                sf.write(denoised_file_path, denoised_segment, sample_rate)  # Use soundfile to save WAV files

                print(f"Denoised segment {label} saved as {denoised_file_name}")
            else:
                print(f"Warning: Segment for {label} in {file} is empty. Skipping this segment.")

print("Denoising complete.")
